## Coding Exercise #0612

In [ ]:
import numpy as np
import warnings
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition  import TruncatedSVD
warnings.filterwarnings('ignore')

### 1. Análisis semántico latente (LSA):

In [ ]:
# Los datos.
my_docs = ["The economic slowdown is becoming more severe",
           "The movie was simply awesome",
           "I like cooking my own food",
           "Samsung is announcing a new technology",
           "Machine Learning is an example of awesome technology",
           "All of us were excited at the movie",
           "We have to do more to reverse the economic slowdown"]

#### 1.1. Crear la representación TF IDF:
Argumentos de TfidfVectorizer(): <br>
- *max_features* : número máximo de características (palabras distintas). <br>
- *min_df* : El DF mínimo. El valor entero significa recuento y el número real (0 ~ 1) significa proporción. <br> 
- *max_df* : El DF mínimo. El valor entero significa recuento y el número real (0 ~ 1) significa proporción. Ayuda a filtrar las palabras vacías. <br> 

In [ ]:
my_docs = [x.lower() for x in my_docs]

In [ ]:
my_stop_words = ['us', 'like']

In [ ]:
vectorizer = TfidfVectorizer(max_features = 15, min_df = 1, max_df = 3, stop_words = stopwords.words('english') + my_stop_words)
X = vectorizer.fit_transform(my_docs).toarray()              

In [ ]:
# Tamaño de X (=m x n). m = número de documentos = 7 & n = número de características.
X.shape

In [ ]:
# VMostrar las características.
features = vectorizer.get_feature_names()
print(features)

#### 1.2. Aplicar SVD truncado:

In [ ]:
n_topics = 4
svd = TruncatedSVD(n_components=n_topics, n_iter=100)
svd.fit(X)

In [ ]:
# Obtener la matriz V^t. 
vt = svd.components_
vtabs = np.abs(vt)

In [ ]:
# Comprobar el tamaño de V^t. 
vt.shape

#### 1.3. De cada tema, extraer las características principales:

In [ ]:
n_top = 3
for i in range(n_topics):
    topic_features = [features[idx] for idx in np.argsort(-vtabs[i,:])]   # argsort() muestra el índice ordenado.
    topic_features_top = topic_features[0:n_top]
    if i == 0:
        topic_matrix = [topic_features_top]                    # list의 list 만들 준비!
    else:
        topic_matrix.append(topic_features_top) 

In [ ]:
# Muestre las características principales de cada tema.
topic_matrix

In [ ]:
# En vista de las características principales, podemos nombrar los temas.
topic_names = ['Economy', 'Movie','Technology', 'Cuisine']

#### 1.4. Etiquetar cada documento con el tema principal:

In [ ]:
n_docs = len(my_docs)
for i in range(n_docs):
    score_pick = 0
    topic_pick = 0
    tokennized_doc = nltk.word_tokenize(my_docs[i])
    for j in range(n_topics):
        found = [ x in topic_matrix[j] for x in tokennized_doc ] 
        score = np.sum(found)
        if (score > score_pick):
            score_pick = score
            topic_pick = j
    print("Document " + str(i+1) + " = " + topic_names[topic_pick])

**NOTA**: Podemos notar algunas inexactitudes.